In [1]:
import warnings
warnings.filterwarnings("ignore")

In [2]:
import pandas as pd
from src.paths import TRANSFORMED_DATA_DIR

df = pd.read_parquet(TRANSFORMED_DATA_DIR / 'tabular_data.parquet')
df

,rides_previous_672_hour,rides_previous_671_hour,rides_previous_670_hour,rides_previous_669_hour,rides_previous_668_hour,rides_previous_667_hour,rides_previous_666_hour,rides_previous_665_hour,rides_previous_664_hour,rides_previous_663_hour,...,rides_previous_7_hour,rides_previous_6_hour,rides_previous_5_hour,rides_previous_4_hour,rides_previous_3_hour,rides_previous_2_hour,rides_previous_1_hour,pickup_hour,pickup_location_id,target_rides_next_hour
0,0.0,0.0,0.0,0.0,1.0,1.0,0.0,2.0,0.0,0.0,...,2.0,0.0,1.0,0.0,0.0,0.0,0.0,2022-01-29,1,0.0
1,0.0,0.0,0.0,0.0,0.0,4.0,1.0,2.0,1.0,2.0,...,0.0,0.0,0.0,0.0,1.0,0.0,0.0,2022-01-30,1,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2.0,0.0,0.0,...,0.0,1.0,2.0,0.0,0.0,0.0,0.0,2022-01-31,1,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,1.0,1.0,...,2.0,1.0,0.0,1.0,1.0,0.0,0.0,2022-02-01,1,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,2022-02-02,1,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
88289,3.0,0.0,2.0,3.0,2.0,3.0,13.0,8.0,9.0,9.0,...,6.0,5.0,3.0,1.0,6.0,1.0,3.0,2022-12-27,265,3.0
88290,6.0,4.0,0.0,0.0,2.0,0.0,14.0,7.0,8.0,4.0,...,4.0,2.0,1.0,2.0,2.0,2.0,8.0,2022-12-28,265,1.0
88291,7.0,2.0,3.0,4.0,7.0,4.0,10.0,9.0,7.0,11.0,...,2.0,3.0,5.0,1.0,1.0,0.0,8.0,2022-12-29,265,3.0
88292,6.0,5.0,4.0,3.0,0.0,3.0,11.0,12.0,9.0,10.0,...,3.0,3.0,1.0,2.0,0.0,1.0,2.0,2022-12-30,265,7.0


In [4]:
from datetime import datetime
from src.data_split import train_test_split

X_train, y_train, X_test, y_test = train_test_split(
    df, 
    cutoff_date=datetime(2022, 6, 1, 0, 0, 0),
    target_column_name='target_rides_next_hour'
)

print(f'{X_train.shape=}')
print(f'{y_train.shape=}')
print(f'{X_test.shape=}')
print(f'{y_test.shape=}')

X_train.shape=(32226, 674)
y_train.shape=(32226,)
X_test.shape=(56068, 674)
y_test.shape=(56068,)


In [28]:
import numpy as np
from sklearn.model_selection import KFold, TimeSeriesSplit
from sklearn.pipeline import make_pipeline
from sklearn.metrics import mean_absolute_error
import optuna

from src.model import get_pipeline

def objective(trial: optuna.trial.Trial) -> float:
    """
    Given a set of hyper-parameters, it trains a model and computes an average
    validation error based on TimeSeriesSplit
    """
    # pick hyper-parameters
    hyperparams = {
        "metric": 'mae',
        "verbose": -1,
        "num_leaves": trial.suggest_int("num_leaves", 2, 256),
        "feature_fraction": trial.suggest_float("feature_fraction", 0.2, 1.0),
        "bagging_fraction": trial.suggest_float("bagging_fraction", 0.2, 1.0),
        "min_child_samples": trial.suggest_int("min_child_samples", 3, 100),
    }
    
    tss = TimeSeriesSplit(n_splits=2)
    scores = []
    for train_index, val_index in tss.split(X_train):
        
        # split data for training and validation
        X_train_, X_val_ = X_train.iloc[train_index, :], X_train.iloc[val_index, :]
        y_train_, y_val_ = y_train.iloc[train_index], y_train.iloc[val_index]
        
        # train the model
        pipeline = get_pipeline(**hyperparams)
        pipeline.fit(X_train_, y_train_)
        
        # evaluate the model
        y_pred = pipeline.predict(X_val_)
        mae = mean_absolute_error(y_val_, y_pred)
        
        scores.append(mae)
        
    # Return the mean score
    return np.array(scores).mean()

In [36]:
study = optuna.create_study(direction="minimize")
study.optimize(objective, n_trials=20)

[I 2023-12-15 11:58:06,073] A new study created in memory with name: no-name-dcc7f94e-3f47-441b-9d8c-222d4ad66602
[I 2023-12-15 11:58:16,829] Trial 0 finished with value: 3.388898235245166 and parameters: {'num_leaves': 105, 'feature_fraction': 0.5212196258335997, 'bagging_fraction': 0.9961185323767476, 'min_child_samples': 51}. Best is trial 0 with value: 3.388898235245166.
[I 2023-12-15 11:58:25,428] Trial 1 finished with value: 3.3720096155468653 and parameters: {'num_leaves': 172, 'feature_fraction': 0.2798962800234479, 'bagging_fraction': 0.729350237005828, 'min_child_samples': 14}. Best is trial 1 with value: 3.3720096155468653.
[I 2023-12-15 11:58:43,892] Trial 2 finished with value: 3.3525115963138714 and parameters: {'num_leaves': 167, 'feature_fraction': 0.9291871270883856, 'bagging_fraction': 0.5493454580976961, 'min_child_samples': 35}. Best is trial 2 with value: 3.3525115963138714.
[I 2023-12-15 11:58:48,640] Trial 3 finished with value: 3.6635923623352245 and parameters:

In [37]:
best_params = study.best_trial.params
print(f'{best_params=}')

best_params={'num_leaves': 67, 'feature_fraction': 0.7536411141451993, 'bagging_fraction': 0.32793187690951753, 'min_child_samples': 31}


In [38]:
pipeline = get_pipeline(**best_params)
pipeline.fit(X_train, y_train)

[LightGBM] [Warning] feature_fraction is set=0.7536411141451993, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7536411141451993
[LightGBM] [Warning] bagging_fraction is set=0.32793187690951753, subsample=1.0 will be ignored. Current value: bagging_fraction=0.32793187690951753
[LightGBM] [Warning] feature_fraction is set=0.7536411141451993, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7536411141451993
[LightGBM] [Warning] bagging_fraction is set=0.32793187690951753, subsample=1.0 will be ignored. Current value: bagging_fraction=0.32793187690951753
[LightGBM] [Info] Auto-choosing col-wise multi-threading, the overhead of testing was 0.133356 seconds.
You can set `force_col_wise=true` to remove the overhead.
[LightGBM] [Info] Total Bins 154527
[LightGBM] [Info] Number of data points in the train set: 32226, number of used features: 675
[LightGBM] [Info] Start training from score 11.703562


Pipeline(steps=[('functiontransformer',
                 FunctionTransformer(func=<function average_rides_last_4_weeks at 0x7f3ac42a3060>)),
                ('temporalfeaturesengineer', TemporalFeaturesEngineer()),
                ('lgbmregressor',
                 LGBMRegressor(bagging_fraction=0.32793187690951753,
                               feature_fraction=0.7536411141451993,
                               min_child_samples=31, num_leaves=67))])

In [39]:
predictions = pipeline.predict(X_test)
test_mae = mean_absolute_error(y_test, predictions)
print(f'{test_mae=:.4f}')

[LightGBM] [Warning] feature_fraction is set=0.7536411141451993, colsample_bytree=1.0 will be ignored. Current value: feature_fraction=0.7536411141451993
[LightGBM] [Warning] bagging_fraction is set=0.32793187690951753, subsample=1.0 will be ignored. Current value: bagging_fraction=0.32793187690951753
test_mae=2.5816


In [52]:
from src.plot import plot_one_sample

plot_one_sample(
    example_id=2979,
    features=X_test,
    targets= y_test,
    predictions=pd.Series(predictions)
)

In [42]:
plot_one_sample(
    example_id=3989,
    features=X_test,
    targets=y_test,
    predictions=pd.Series(predictions)
)